In [3]:

import cv2
from cv2 import dnn_superres
import os
import numpy as np
# TODO 绘图
def showBenchmark(imgs, titles, psnrValues, ssimValues):
    # 绘图
    for i in range(0, len(imgs)):
        # 标题绘图
        cv2.putText(imgs[i], titles[i], (10, 15), cv2.FONT_HERSHEY_PLAIN, 1.3,(1, 1, 255), 1, cv2.LINE_AA)
        # psnr值
        if i>0:
         
         cv2.putText(imgs[i], "PSNR: " + str(psnrValues[i-1]), (10, 30), cv2.FONT_HERSHEY_PLAIN, 1.3,
                    (1, 1, 255), 1, cv2.LINE_AA)
        # ssim值
         cv2.putText(imgs[i], "SSIM: " + str(ssimValues[i-1]), (10, 45), cv2.FONT_HERSHEY_PLAIN, 1.3,
                    (1, 1, 255), 1, cv2.LINE_AA)
    # 图片拼接展示
    img = np.vstack([np.hstack([imgs[0],imgs[1],imgs[2]])])
    allimg_path = "./Set5/allwoman.png"
    cv2.imshow("Quality benchmark", img)
    cv2.imwrite(allimg_path, img)
    cv2.waitKey(0)
# TODO 图像质量评估
def getQualityValues(upsampled, orig):
    psnr = cv2.PSNR(upsampled, orig)
    q, _ = cv2.quality.QualitySSIM_compute(upsampled, orig)
    ssim = (q[0] + q[1] + q[2]) / 3
    return round(psnr, 3), round(ssim, 3)
def main():
    # 图片路径
    img_path = "./Set5/original/woman.png"
    # 算法名称 edsr, espcn, fsrcnn or lapsrn
    algorithm = "espcn"
    # 模型路径，根据算法确定
    model = "./model/ESPCN_x4.pb"
    # 放大系数
    scale = 4
    psnrValues = []
    ssimValues = []
    img = cv2.imread(img_path)
    if img is None:
        print("Couldn't load image: " + str(img_path))
    # 缩小图像，以实现基准质量测试
    img_downscaled = cv2.resize(img, None, fx=1.0 / scale, fy=1.0 / scale,interpolation=cv2.INTER_CUBIC )
    saveFile = "./Set5/downscaled/woman_d.png"  # 保存文件的路径
    cv2.imwrite(saveFile, img_downscaled)  # 保存图像文件
    # 超分模型初始化
#     sr =cv2.dnn_superres.DnnSuperResImpl_create()
    sr = cv2.dnn_superres.DnnSuperResImpl.create()
    # Read and set the dnn model
    # 读取和设定模型
    sr.readModel(model)
    sr.setModel(algorithm, scale)
    # 放大图像
    img_new = sr.upsample(img_downscaled)
    # DL MODEL
    # 获得模型质量评估值
    psnr, ssim = getQualityValues(img_new, img)
    psnrValues.append(psnr)
    ssimValues.append(ssim)
    print(sr.getAlgorithm() + "\n")
    print("PSNR: " + str(psnr) + " SSIM: " + str(ssim) + "\n")
    print("-" * 50)
    # INTER_CUBIC - 三次样条插值放大图像
    bicubic = cv2.resize(img_downscaled, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
    psnr, ssim = getQualityValues(bicubic, img)
    psnrValues.append(psnr)
    ssimValues.append(ssim)
    print("Bicubic \n")
    print("PSNR: " + str(psnr) + " SSIM: " + str(ssim) + "\n")
    print("-" * 50)
    imgs = [img,img_new, bicubic]
    titles = ["Original","Espcn", "Bicubic"]
    showBenchmark(imgs, titles, psnrValues, ssimValues)
if __name__ == '__main__':
    main()


espcn

PSNR: 25.49 SSIM: 0.84

--------------------------------------------------
Bicubic 

PSNR: 24.446 SSIM: 0.817

--------------------------------------------------
